In [3]:
from selenium import webdriver
import time



# login screen move
def login_screen_url() :
    global driver
    driver = webdriver.Chrome('C:\/chromedriver.exe') # 제어하기 위해 변수 선언
    driver.maximize_window() ### 창의 해상도 크기 최대화 (전체화면) ###
    url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb" # 이동할 링크주소를 변수에 저장
    driver.get(url) # 해당 링크로 이동

    return driver



# user info (login)
def user_info():
    
    # ID
    xpath = "//*[@id='user_email']" 
    input_search = driver.find_element_by_xpath(xpath).send_keys("injekim97@catholic.ac.kr")


    # password
    xpath2 = "//*[@id='user_password']" 
    input_search = driver.find_element_by_xpath(xpath2).send_keys("k5311810!!")


    # button click
    xpath3 = "//*[@id='signInSignInCon']/div[2]/div/section[2]/fieldset/button" # x 버튼, //button[@class='closeBtn']에다가 "" 로감싸야함
    login = driver.find_element_by_xpath(xpath3).click()


    
# Enterprise input
def Enterprise():
    # 기업명 이름
    xpath4 = "//*[@id='search_bar_search_query']" 
    Enterprise_input = driver.find_element_by_xpath(xpath4).send_keys("카페24")
    
    # 검색 버튼 클릭
    search = "//*[@id='search_form']/div/button"
    btn_search = driver.find_element_by_xpath(search).click()

    # 검색한 기업 이름 클릭
    xpath5 = "//*[@id='mainContents']/div[1]/div/div[2]/div[1]/div[1]/a"
    Enterprise_name = driver.find_element_by_xpath(xpath5).click()
    time.sleep(2)
    
    
    # 검색한 기업의 면접 리뷰 클릭
    xpath6 = " //*[@id='viewCompaniesMenu']/ul/li[5]/a"
    review = driver.find_element_by_xpath(xpath6).click()
    time.sleep(2)
    
    scroll_down = driver.execute_script("window.scrollTo(0, 650)") 

    
    
    # 검색한 기업의 직종 select box 클릭
    xpath7 = "//*[@id='occupation_select']"
    job = driver.find_element_by_xpath(xpath7).click()
    time.sleep(2)
    
    # select box 를 클릭 해서 IT/인터넷 클릭 (Value는 source 로 open 후 확인)
    select_box = driver.find_element_by_xpath("//select/option[@value='11600']").click()
    time.sleep(2)

    
    
    # select box로 인해 스크롤 초기화 -> 다시 내려주기 
    scroll_down = driver.execute_script("window.scrollTo(0, 650)") 

    
    # 검색한 기업의 직급 select box 클릭
    xpath8 = "//*[@id='job_rank_select']"
    job_rank = driver.find_element_by_xpath(xpath8).click()
    time.sleep(2)
    
    # select box 를 클릭 해서 사원-대졸 클릭 (Value는 source 로 open 후 확인)
    select_box = driver.find_element_by_xpath("//select/option[@value='1']").click()
    time.sleep(2)
    
    
    # select box로 인해 스크롤 초기화 -> 다시 내려주기 
    scroll_down = driver.execute_script("window.scrollTo(0, 650)") 

    
    
# 본격적인 crawling
def craw() :
    data = []
    idx = 1

    print("★★★★★ 기출 면접 data crawling 시작합니다 ★★★★★")

    try : 
        
        # Pagination 1~47
        for j in range(1,48):
            url = f"https://www.jobplanet.co.kr/companies/23717/interviews_by_filter?by_occupation=11600&by_job_rank=1&by_success=&page={j}"
            driver.get(url)
            
            
            # 해당 page의 i번째 게시글 데이터 추출  
            for i in range(1,6) :
                
                theme = driver.find_element_by_xpath(f"//*[@id='viewInterviewsList']/div/div/section[{i}]/div/div[2]/div/div[1]/h2")    
                question = driver.find_element_by_xpath(f"//*[@id='viewInterviewsList']/div/div/section[{i}]/div/div[2]/div/dl/dd[1]/span")
                question_result = driver.find_element_by_xpath(f"//*[@id='viewInterviewsList']/div/div/section[{i}]/div/div[2]/div/div[2]/div/dl/dd[1]")

                  
              # replace \n 제거
                res_theme = theme.text.replace("\n","")
                res_question = question.text.replace("\n","")
                res_question_result = question_result.text.replace("\n","")

                
                # result 결과
                data.append(res_theme)
                data.append(res_question)
                data.append(res_question_result)

        
                
                print(f"{idx}번째 게시글")
                print()
                
                print("주제:",data[0])
                print()
                
                
                # ★★★ review_data가 있을 경우에만 ★★★
                try :
                    temp = []
                    review_data = driver.find_element_by_xpath(f"//*[@id='viewInterviewsList']/div/div/section[{i}]/div/div[2]/dl/dd[2]")
                    res_review_data = review_data.text.replace("\n","")
                    temp.append(res_review_data)
                    print("면접일자:",temp[0])
                    print()
                    
                except :
                    pass
                
                
                print("면접질문:",data[1])
                print()
                
                
                   
                # ★★★★★ 면접답변 혹은 면접느낌이 있을 경우에만 ★★★★★
                try :
                    temp2 = []
                    question2 = driver.find_element_by_xpath(f"//*[@id='viewInterviewsList']/div/div/section[{i}]/div/div[2]/div/dl/dd[2]")
                    res_question2 = question2.text.replace("\n","")
                    data.append(res_question2)
                    print("면접답변 혹은 면접느낌:",temp2[0])
                    print()
                    
                except :
                    pass
                
                
                print("면접결과:",data[2])
                print()
                print("---------------------------------------------------")
                idx += 1
                data = [] # 출력했으니 초기화
                
                
    except Exception as e :
        print("페이지 네이션 오류",e) 
        
        
# --------------------------- main -------------------------
login_screen_url() # jobplanet login screen url 
time.sleep(1)

user_info() # ID & PWD input -> login button click
time.sleep(2)

Enterprise() # 기업명 입력 후 면접 버튼 클릭 후 직군 , 직급 클릭 
craw() # 면접 review data 추출

★★★★★ 기출 면접 data crawling 시작합니다 ★★★★★
1번째 게시글

주제: "1차 면접은 인사팀과 실무 면접관 한 분씩 오셔서 2:1로 진행되었습니다.2차 면접은 CTO와 1:1 면접입니다."

면접일자: 2018/01

면접질문: 1차 면접은 자기소개와 기술 관련 질문이 있었고 직무경험과 같은 질문이 있었습니다.2차 면접도 역시 기술면접과 자기소개 등 면접 질문이 있었습니다.

면접결과: 대기중

---------------------------------------------------
2번째 게시글

주제: "지원하고 3일인가 있다가 연락와서 면접 가능한 날짜 잡고 면접을 보러감긴장하지 않도록 잘 이끌어 주시고 끝나면 고생하셨다며 엘베까지 마중도 나와주셔서 기분 좋게 면접 봤던 기억이 있음"

면접일자: 2018/01

면접질문: 1차 인성 면접은 1:1로 진행했고, 2차는 면접관 두분이 들어오셔서 1:2로 진행간단한 자기소개부터 시작해서 주로 이력서와 자기소개서에 썼던 내용들 위주로 질문함자신이 지원한 직무에 대한 이해는 필수

면접결과: 불합격

---------------------------------------------------
3번째 게시글

주제: "지원후 1주일 지나서 전화로 연락이 옴.면접 시간은 최대한 배려해 줬고 2일 후 면접 보러 감.많이 떨렸는데 면접 전 친절하게 안내해 줘서 좀 풀림."

면접일자: 2017/09

면접질문: 면접관 한분이 들어와서 그동안 했던 프로젝트 등에 대해 질문.이력서에 있는 내용 위주로 질문.이후 회사설명 해주시고 1차 면접은 끝남.대체적으로 편안한 분위기에서 면접이 진행되었고 면접을 보면서 내가 이런 부분에서 준비가 많이 부족했구나를 느낌.ㅠㅠ면접 질문은 대체적으로 평이함.

면접결과: 불합격

---------------------------------------------------
4번째 게시글

주제: "php 개발자 채용이었고,php는 써본적이 없어서 면접 질문이라곤 인성질문

면접결과: 합격

---------------------------------------------------
40번째 게시글

주제: "인사팀 면접 -> 실무팀 면접으로 진행인사팀 1:1실무자(팀장급) 2명 : 지원자 1명"

면접질문: 언어테스트는 기본으로 보고, 야근많은데 괜찮은 지 물어봅니다

면접결과: 합격

---------------------------------------------------
41번째 게시글

주제: "인사팀 면접을 보고 실무팀 면접을 봤습니다.평범한 면접이였습니다"

면접일자: 온라인 지원

면접질문: 회사 지원동기를 물어봄특수한상황에 대한 대응방법 물어봄

면접결과: 합격

---------------------------------------------------
42번째 게시글

주제: "실무진-인사팀 이렇게 진행해요.분위기는 편했고 크게 어려운게 없었어요."

면접일자: 2017/01

면접질문: 실무진 면점 - 직무 관련 경력 유무, 팀워크 경험, 자기소개인사팀 면접 - 복지 설명 및 연봉 협의

면접결과: 합격

---------------------------------------------------
43번째 게시글

주제: "1차에서 인사팀 인터뷰 진행 후 실무진 인터뷰 진행2차는 cto와 1:1면접"

면접일자: 온라인 지원

면접질문: 1.인사팀 면접에서 희망 연봉을 말해야함. 1/13임을 설명해줌2. 실무진 인터뷰에서 프로젝트 관련 질문과 경력 관련내옹을 상세하게 물어봄1차 인터뷰는 이틀뒤에 바로 알려줬음. 1차 인터뷰에서 희망연봉을 말하라거 해서 당황. 1차에서 연봉 높게 부르면 떨어질 수도 있을듯..2차는 c.t.o와 면접. 1차 실무 인터뷰에서 한시간동안 기술관련 질문을 받았는데 2차에서도 1시간동안 기술질문만 받음. 답변이 마음에 안든건지 표정이 계속 안좋고 그래서 그런지 보고나서도 기분이 계속 별로였음.

면접결과: 대기중

---------------------------------------

면접결과: 합격

---------------------------------------------------
78번째 게시글

주제: "면접부터 채용까지 약 2주소요 되었습니다. 메일로 지원했고 메일로 면접제의를 받았으며 채용전화로 합격을 통보받았습니다"

면접질문: Db에 관한 질문, 프로젝트 진행경험, 영어실력을 물었습니다

면접결과: 합격

---------------------------------------------------
79번째 게시글

주제: "코딩테스트를 본 후 1차면접 2차면접(cto) 면접으로 나눠집니다."

면접일자: 2019/04

면접질문: 렌더링에 대해 아는 것을 설명하시오.

면접결과: 합격

---------------------------------------------------
80번째 게시글

주제: "좋았습니다. 너무 편하게 해주셔서 몸둘바를 몰랐습니다."

면접질문: 성격의 장단점 이 무엇인지 알려달라고 하였고, 위기를 극복한 사례

면접결과: 대기중

---------------------------------------------------
81번째 게시글

주제: "1.전화로 면접 일정 픽스2.면접진행(1회 진행, 팀장면접)3.통보(땡땡님과 같이 일하고 싶습니다.기분이 좋아짐 ㅎ)"

면접일자: 2015/01

면접질문: mysql 관련 질문(즉 업무에 대한 질문이 많았음)1.mysql 의 장점2.innodb와 myisam 차이점

면접결과: 합격

---------------------------------------------------
82번째 게시글

주제: "서류-코딩테스트-직무면접 여기서 탈락이후엔 임원 면접-교육-입사 과정이었습니다.면접관분들은 친절했고 편안한 분위기로 유도해주셨습니다."

면접일자: 2018/06

면접질문: 질문 난이도는 보통이었습니다. 마지막에는 듣고싶었던 질문을 물어보시더군요. 대비했던 질문과 그에 맞는 대답해주시면됩니다.

면접결과: 불합격

---------------

114번째 게시글

주제: "인사면접, 실무면접, CTO면접 순서로 진행하고면접결과는 바로바로 연락 줌"

면접일자: 2018/02

면접질문: 다른 도메인이 세션 공유를 하는 방법restful api암호화 방법

면접결과: 합격

---------------------------------------------------
115번째 게시글

주제: "서류전형 이후에 타 교육기관에서 필기시험을 치고 필기시험 합격하면 1차면접이 진행됩니다. 전화로 연락이 오고 1차 면접 이후에 2차 면접을 보러오라고 다시 한 번 전화로 연락이 옵니다."

면접일자: 2018/02

면접질문: 1차면접은 필기시험의 연장선과 같이 기술관련 질문들을 받습니다. 지원하는 업무와 관련된 기본에 충실하면 충분히 답할 수 있을 정도의 질문들 입니다. 기본이 가장 중요하다는 것을 다시 한 번 느끼게 되었습니다. 2차 면접은 CTO님과의 1:1 면접인데 대화를 나누듯 지원하는 업무에 관한 전체적인 그림을 그려보는 질문과 인성에 관한 질문입니다. 물론 자기소개는 기본적으로 준비해야 합니다.

면접결과: 불합격

---------------------------------------------------
116번째 게시글

주제: "1차 인사팀 면접2차 실무팀 면접이 모든게 하루만에 이루어짐면접시간 한시간반 정도 소요"

면접일자: 2014/03

면접질문: 업무 관련한 실무적인 내용과 업무역량, 실무 성과등에 관련하여서 물어봄

면접결과: 합격

---------------------------------------------------
117번째 게시글

주제: "2대1의 면접으로 경력을 묻는 질문을 주로 물어봄. 과거 경력 위주의 질문들로 40분가량 진행"

면접일자: 2017/09

면접질문: 업무중 생긴 문제와 그 문제를 주도적으로 해결했던 경험을 이여가 해보세요

면접결과: 불합격

---------------------------------------------------
118번째 

주제: "잡코리아에서 다른직종으로 지원했지만 인사팀에서 맞을것 같은 직종으로 제안받았음1,2차를 동시에 봤으며1차에는 인성면접(압박없음)2차에는 PT면접과 기술면접을 보았지만기술면접에 대한 준비가 없었던것 같아 마음의 준비를 하고있음"

면접일자: 2016/08

면접질문: 1차살아오면서 했던 일을 기준으로 자기소개성격의 장단점무슨 경험을 하였는가2차준비해온 자료로 발표진행트렌젝션에 대한 질문이었으나 답변못함...반응형 웹에 대한 질문이 있었지만 틀리게답변간단한 영어독해하고 요약해서해석SQL인젝션이란 무엇인가웹 해킹에 대해서 배웠는가

면접결과: 대기중

---------------------------------------------------
146번째 게시글

주제: "인사면접과 실무진 면접을 당일날 같이 보았고 실무진 면접은 면접질문 프로세스가 존재함"

면접일자: 2016/07

면접질문: 인사면접은 뻔한 질문들을 했고 실무진면접은 질문프로세스가 존재해서 너무많아서 기억안남 기술질문은 별로 안함

면접결과: 불합격

---------------------------------------------------
147번째 게시글

주제: "면접은 팀장, 이사님 포함 3분 진행(기술) - 이거저거 물어봄인사팀장님(인사, 연봉조정 면접) - 이건 면접이라고보기엔 그냥 정보 커뮤니케이션최종 대표님 면접 - 가치관등 물어보는듯"

면접일자: 2011/10

면접질문: 자기자랑 해보라함.신기술에 대한 습득경험회사에 대한 피드백 요청

면접결과: 합격

---------------------------------------------------
148번째 게시글

주제: "1차 인사담당자와 1:1 면접 후, 함께 근무할 부서 팀장과 1:1 면접 진행.인사담당자는 매우 편안하게 웃으며 진행하였고, 부서 팀장은 얼굴도 안보고 면접 진행함 그리고 스케쥴있다고 면접은 여기까지라며 내게 모욕감을 주었음..."

면접일자: 2016/02

면접질문: 자기소개, 지원동기, 단점

178번째 게시글

주제: "인사팀, 해당팀장, 대표이사 면접까지 진행.온라인으로 먼저 지원 후 면접 제의 받음."

면접일자: 2010/07

면접질문: 이전 프로젝트에 대한 질문이 많았음.

면접결과: 합격

---------------------------------------------------
179번째 게시글

주제: "입사지원 하고 몇일 안되서 연락오고면접 보고 바로 얼마안되서 합격 연락오고 끝"

면접일자: 2011/05

면접질문: 자신이 했던 업무중 가장 자신있는게 무었입니까?

면접결과: 합격

---------------------------------------------------
180번째 게시글

주제: "1차 인성 2차 실무진 면접을 보았고 실무진 면접만 3번 보았습니다.(필요한 팀에서 계속 면접을 봄)"

면접일자: 2015/07

면접질문: 인성면접은 기존 경력에 대한 내용과 자신의 강점 및 지원동기 등실무진은 기본적인 프로그래밍 질문과 최신 트랜드에 대한 질문을함

면접결과: 불합격

---------------------------------------------------
181번째 게시글

주제: "신입으로 지원시 면접 순서.1. 필기 시험 - 기본적인 개발 언어 문제2. 인사팀&CTO면접 - 인성 면접 및 개발 지식 면접3. 실무진 면접"

면접일자: 2012/11

면접질문: 자기 PR 후 기본적인 질답 형식으로 면접 진행됨.기초적인 개발 지식 질문(DB / 개발 언어 / 형상관리 / 자바스크립트)입사 지원 동기 및 포부를 질문함.

면접결과: 합격

---------------------------------------------------
182번째 게시글

주제: "인사팀 개발팀 실무진 등등 면접보는데 대체적으로 자유롭게 괜찮음"

면접일자: 2012/10

면접질문: 논리적인 문제와 문제해결 능력을 보는 문제를 어떤 방향으로 해결하는지 보십니다

면접결과: 합격

------------------------

213번째 게시글

주제: "서류 접수하고 다음날 연락왔구요.PT발표 및 인성면접 같은 날 진행했습니다.PT발표 준비는 서류 합격전화 온 후 메일로 주제 알려주셨구요.."

면접일자: 온라인 지원

면접질문: 대체로 평이했습니다.

면접결과: 합격

---------------------------------------------------
214번째 게시글

주제: "지원 하루 이틀 뒤 바로 연락옴. 하루에 인사 및 실무 면접 모두 진행함."

면접일자: 온라인 지원

면접질문: 인사팀 면접이 상당히 불쾌했음.경역악화로 퇴사한 이력을 보더니 회사 보는 눈이 없냐 부터 시작해서 사람을 감정적으로 몰고감.연차가 적었을 때라 더 그렇게 느꼈을 수도.실무는 평이했음.

면접결과: 불합격

---------------------------------------------------
215번째 게시글

주제: BEST"사내직원 추천으로 1차 면접 잡힘기술면접 전에 HR팀과의 짤막한 면접이 있음. 이때 연봉 정보에 대해서 논의함.기술면접은 CTO,팀장과 함께 2:1로 진행됐음기술면접시 이력사항에 대한 기술적인 부분을 디테일하게 물어봄화요일에 1차 면접 보고, 수요일에 합격 전화옴목요일 2차 임원 면접 잡힘.2차 면접은 최신 업계 동향이나 트렌드에 관련된 부분을 주로 물어봄이때 직접적으로 물어보지 않고, 숨겨진 뜻이 있는식으로 물어보므로CEO가 질문한것을 그대로 받아드리지 말고 진짜 의도가 무엇인지 잘 파악하고 대답하는게 중요함"

면접질문: 크게 어려운거 없었음.

면접결과: 합격

---------------------------------------------------
216번째 게시글

주제: BEST"면접은 2번 봄. 인사팀, 사장님. 하지만 한번 떨어지고 두번 면접봐서 인사팀만 봄. 사장님 면접 생략.면접은 가능한 한 웃으면서 편안한 분위기 유지해 줌. 두번째 면접봤을 때 희망연봉 물어봄.회사에 대한 생각보다는 기술적으로 얼마나 잘 알고 있는지, 평소 성격은 